#  Tanzania Water Pump Failure Predictions - Competition

## <font color=green>Implement a Predictive Model to help solve this problem and submit to DRIVENDATA competition</font>



>  **This is a WORK-IN-PROGRESS -   10/26/2022 - please check back for the final version...**

...


Can you predict which water pumps are faulty?

Using data from Taarifa and the Tanzanian Ministry of Water, 
can you predict which pumps are functional, which need some repairs, 
and which don't work at all? Predict one of these three classes
based on a number of variables about what kind of pump is operating, 
when it was installed, and how it is managed. 

A smart understanding of which waterpoints will fail can improve maintenance operations 
and ensure that clean, potable water is available to communities across Tanzania.

...

Hope this simple example of an XGBoost predictive model helps!

All the best,
Mike Pastor


**Competition Home Page:**  
[DRIVENDATA - Data science competitions
to build a better world](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/)



#  First, let's load all of the necessary Python libraries
#    ( Note: these need to be installed on the Python instance running the NoteBook)


In [27]:


import pandas as pd # load and manipulate data and for One-Hot Encoding
import numpy as np # calculate the mean and standard deviation
from datetime import datetime
import scipy.stats as stats

import xgboost as xgb # XGBoost stuff

from sklearn.model_selection import train_test_split # split  data into training and testing sets
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer # for scoring during cross validation
from sklearn.model_selection import GridSearchCV # cross validation
from sklearn.metrics import confusion_matrix # creates a confusion matrix
from sklearn.metrics import plot_confusion_matrix # draws a confusion matrix


#################################################################
#  Track the overall time for training & submission preparation
#
global_now = datetime.now()
global_current_time = global_now.strftime("%H:%M:%S")
print("##############  Starting up...  - Current Time =", global_current_time)


##############  Starting up...  - Current Time = 11:51:23


#  Now, let's open the Datasets 


In [28]:

# load TRAIN datasets ###########################################################
#
train_X = pd.read_csv('./TrainingSetValues.csv')
train_Y = pd.read_csv('./TrainingSetLABELS.csv')

# Merge the Y onto the X to be sure they are matched
#  This adds 'status_group' LABEL onto each X example row
#
train_X = train_X.merge( train_Y, how='inner', on='id' )



#########################################################
# replacing Y  string values with numerics
#    This is becomes part of our Tanzania Data Dictionary
#
train_X['status_group'].replace(\
    ['functional', 'functional needs repair', 'non functional'],\
    [0, 1, 2], inplace=True)

# Used the synchronized - quantitative  LABEL for Y
train_Y = pd.DataFrame( train_X['status_group'], columns=['status_group'] )


########################################################
#  Also  Load the TEST dataset which is used for the Submission preparation
#
test_X = pd.read_csv('./TestSetValues.csv')


#  Take a subset for testing
train_X = train_X[ [    'amount_tsh', 'gps_height', 'longitude', 'latitude', 'population', 'construction_year' ] ]
#                         # 'latitude', 'longitude', 'basin', 'region_code', 'district_code', 'population', \
#                         # 'scheme_name', 'scheme_management', 'construction_year', 'quantity',   'payment', \
#                         # 'extraction_type', 'extraction_type_class', 'source', 'source_type', \
#                         'management', 'waterpoint_type'  ]  ]


# Save IDs for Submission file preparation below
test_X_Saved_IDs = test_X[ [  'id' ] ]

test_X = test_X[ [      'amount_tsh', 'gps_height', 'longitude', 'latitude', 'population', 'construction_year' ] ]
                        # 'latitude', 'longitude', 'basin', 'region_code', 'district_code', 'population', \
                        # 'scheme_name', 'scheme_management', 'construction_year', 'quantity',   'payment', \
                        # 'extraction_type', 'extraction_type_class', 'source', 'source_type', \
                        # 'management', 'waterpoint_type'  ]  ]


print( train_X.dtypes )


print('Datasets Loaded & Merged...   Y - X ', train_Y.shape, train_X.shape )


amount_tsh           float64
gps_height             int64
longitude            float64
latitude             float64
population             int64
construction_year      int64
dtype: object
Datasets Loaded & Merged...   Y - X  (59400, 1) (59400, 6)


# Okay, now let's massage our data into a readable format...


In [29]:
##############################################################
#   Let's work on the train_X  CATEGORICAL columns
#       - Same for TEST

print( 'before get_dummies Train_X ==', train_X.shape )
print( 'before get_dummies test_X ==', test_X.shape )
train_X = pd.get_dummies( train_X )
test_X = pd.get_dummies( test_X )

# Align the number of features across test sets based on train dataset
train_X, test_X = train_X.align(test_X, join='left', axis=1, fill_value=0 )

print( 'AFTER  Train_X ==', train_X.shape )
print( 'AFTER test_X ==', test_X.shape )


###############################################################
# define list  categorical variables (columns)
categorical = list(train_X.select_dtypes('object').columns)
print(f"Categorical variables (columns) are: {categorical}")

# define list   numerical variables (columns)
numerical = list(train_X.select_dtypes('number').columns)
print(f"Numerical variables (columns) are: {numerical}")

# Save the list of Features here for later reporting use
#
feature_list = train_X.columns


#################################################################
#   Clean ampersands from  2 features
#
#  Bad characters in feature name:   'installer', 'funder',

#  train_X.installer = train_X.installer.str.replace('\W', ' ', regex=True)
#   train_X.funder = train_X.funder.str.replace('\W', ' ', regex=True)

#   print( '#####   Replaced special characters in features ' )



before get_dummies Train_X == (59400, 6)
before get_dummies test_X == (14850, 6)
AFTER  Train_X == (59400, 6)
AFTER test_X == (14850, 6)
Categorical variables (columns) are: []
Numerical variables (columns) are: ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population', 'construction_year']


#  Now split the TRAINING dataset into a VALIDATION set also
##   This allows us to take measurements with a formal submission


In [30]:

####################################################################################
# Create a split Training and new Validation from the  merged TRAIN dataset
#
X_TRAIN_SplitDF, X_VALIDATE_SplitDF,  Y_TRAIN_SplitDF, Y_VALIDATE_SplitDF =\
    train_test_split( train_X,  train_Y, test_size=0.20 )

print(f"***********  Train X  set is {len(X_TRAIN_SplitDF)} rows  and Validation set is {len(X_VALIDATE_SplitDF)} rows ")
print(f"***********  Train Y  set is {len(Y_TRAIN_SplitDF)} rows  and Validation set is {len(Y_VALIDATE_SplitDF)} rows ")


***********  Train X  set is 47520 rows  and Validation set is 11880 rows 
***********  Train Y  set is 47520 rows  and Validation set is 11880 rows 


# Let's start assembling the XGBoost tree model...


In [31]:
################################################################################################

print(" STARTING   XGBoost  Model... ")


from xgboost import XGBClassifier
from xgboost import XGBRegressor       # Regression version of XGBoost

#  enable_categorical=False

model = XGBClassifier()

model.fit( X_TRAIN_SplitDF, Y_TRAIN_SplitDF )

print( "Model is Fit and ready to predict...  - ",model )


 STARTING   XGBoost  Model... 
Model is Fit and ready to predict...  -  XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)


#  Now let's start our performance evaluation...
#  How well are we really predicting here??



In [32]:

###################################################################################
#  Performance & Evaluation of the Model
#


def AssessErrors(model,X,y):

    # Use X to predict on 'model'

    print("\n##### PREDICT from X on model...", )
    predictions = model.predict(X)

    # print("\n##### PREDICTED:  ", predictions )

    # yhat = np.argmax(predictions, axis=1)
    yhat = predictions

    # print("yhat COUNT == ", len( yhat ))
    #  print("y COUNT == ", y.shape )
    # print("y head == ", y.head)

    #  For Each Y ...
    #    Also compute the research statistics - truePositive, etc.
    #
    ctr = 0
    origPositivityRate = predPositivityRate = errors = 0
    truePositive = trueNegative = falsePositive = falseNegative = 0
    df = y.reset_index()  # make sure indexes pair with number of rows

    for index, yp in y.iterrows():

        if ( yp['status_group'] == yhat[ctr] ):
            if yp['status_group'] > 0:
                truePositive = truePositive + 1
            else:
                trueNegative = trueNegative + 1
        else:
            errors = errors + 1   # overall error rate

            if yp['status_group'] > 0:
                falsePositive = falsePositive + 1
            else:
                falseNegative = falseNegative + 1

        if (yp['status_group'] > 0):
            origPositivityRate = origPositivityRate + 1
        if ( yhat[ctr] > 0 ):
            predPositivityRate = predPositivityRate + 1

        ctr = ctr + 1

    print('Precision/Recall Stats...TP first\n')
    print (  truePositive, falsePositive)
    print(  falseNegative, trueNegative)

    precision = truePositive / ( truePositive + falsePositive )
    recall = truePositive / (truePositive + falseNegative )
    print( 'PRECISION == ', precision, '  -  RECALL == ', recall )
    print('ORIG Positivity % == ', (origPositivityRate / ctr) , ' -   PREDICTION Positivity % == ', predPositivityRate / ctr  )

    #
    # doo = yhat != y[:,0]
    #  get only the predictions (yhat) that don't match Groundtruth (y)
    # alt_idxs = np.where(yhat != y[:,0])[0]
    # alt_idxs = np.where(yhat != y)
    # alt_idxs = np.where(yhat == y[:, 0])[0]

    # Return total number of errors
    return(errors)





# First let's test Training against itself...


In [33]:

###################################################################
#  Calculate our ERROR rate on the TRAINING dataset
#
tmp1 = AssessErrors(model,X_TRAIN_SplitDF,Y_TRAIN_SplitDF)
len1 = len(X_TRAIN_SplitDF)
print( f"TRAIN DATASET provides {tmp1} errors out of {len1} predictions  \nfor { round(((len1-tmp1)/len1), 6) } Accuracy and { round((tmp1/len1), 6) } ERROR Rate")





##### PREDICT from X on model...
Precision/Recall Stats...TP first

12851 8782
2961 22926
PRECISION ==  0.5940461332223917   -  RECALL ==  0.8127371616493803
ORIG Positivity % ==  0.455239898989899  -   PREDICTION Positivity % ==  0.3477483164983165
TRAIN DATASET provides 11743 errors out of 47520 predictions  
for 0.752883 Accuracy and 0.247117 ERROR Rate


#  Do the same for our VALIDATION dataset...



In [34]:
###################################################################
#  Calculate our internal ERROR rate on the VALIDATION dataset
#

tmp1 = AssessErrors(model,X_VALIDATE_SplitDF,Y_VALIDATE_SplitDF)
len1 = len(X_VALIDATE_SplitDF)
print( f"VALIDATION DEV SET provides {tmp1} errors out of {len1} predictions \nfor { round(((len1-tmp1)/len1), 6) } Accuracy and { round((tmp1/len1), 6) } ERROR Rate" )




##### PREDICT from X on model...
Precision/Recall Stats...TP first

2913 2595
1016 5356
PRECISION ==  0.5288671023965141   -  RECALL ==  0.7414100279969458
ORIG Positivity % ==  0.4636363636363636  -   PREDICTION Positivity % ==  0.3466329966329966
VALIDATION DEV SET provides 3611 errors out of 11880 predictions 
for 0.696044 Accuracy and 0.303956 ERROR Rate


#  Now we can predict on the provided TEST dataset & prepare the competition Submission file


In [35]:

#########################################################################
#  PREDICT from the TEST set now to prepare the Submission file
#

print("\n##### PREDICT from  TEST   on model for Submission file...")
predictions = model.predict(test_X)  # prediction

predNumpy = pd.DataFrame( predictions )
print("TEST   PREDICTION  Yields  shape ==  ", predNumpy.shape )
# print(f" PREDICTION Yields ==  \n", predNumpy.head() )




##### PREDICT from  TEST   on model for Submission file...
TEST   PREDICTION  Yields  shape ==   (14850, 1)


#  Now write the actual Submission file in the specified format...



In [36]:

#####################################################################
# Now write the submission file in the required format
#
submissionDF = pd.DataFrame(columns =['id', 'status_group'])

# submissionDF['id'] = test_X['id']
submissionDF['id'] = test_X_Saved_IDs

#   For  Neural Networks -
#   Determine the highest prediction %
#   on each row  for the final submission prediction
# submissionDF['status_group'] = np.argmax(predictions, axis=-1 )

#  XGBoost returns the target  as a single number per X instance
#
submissionDF['status_group'] = predictions

# stats
#
positiveList = np.where(submissionDF['status_group'] > 0)
positiveCount = np.array( positiveList )
print("Prediction POSITIVITY Count == ", positiveCount.size, \
      "  Rate == ", ( positiveCount.size / len(submissionDF) ) )

#    Test random entries for Baseline (Accuracy== .3318  on DD )
#       submissionDF['status_group'] =random.randint(3, size=(len(predictions)) )

# Return the numbers to strings per the submission rules
submissionDF['status_group'].replace([0, 1, 2], ['functional', 'functional needs repair', 'non functional'],
                         inplace=True)

# Write the actual file to disk
submissionDF.to_csv( "tanzania_submission.csv", index=False )
print( "\nWrote Submission file (tanzania_submission.csv) with Shape == ", submissionDF.shape )



Prediction POSITIVITY Count ==  5025   Rate ==  0.3383838383838384

Wrote Submission file (tanzania_submission.csv) with Shape ==  (14850, 2)


#  Mission accomplished!


In [37]:

# Mission Complete!
##################################################################################
global_later = datetime.now()
print("#####   Tanzania Water Pump Predictions - Total EXECUTION Time =", (global_later - global_now) )



#####   Tanzania Water Pump Predictions - Total EXECUTION Time = 0:01:04.493564


In [ ]:
#  mlp
